In [ ]:
import os
import cv2
import glob
import pickle
import time
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
from typing import List, Tuple, Union
from argparse import ArgumentParser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dataset 준비

In [ ]:
!unzip -qn /content/drive/MyDrive/train_crop.zip -d /content/data/

#필요한 패키지 다운

In [ ]:
!git clone https://github.com/hhj1897/face_detection.git

Cloning into 'face_detection'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 300 (delta 41), reused 39 (delta 39), pack-reused 250
Receiving objects: 100% (300/300), 81.19 MiB | 11.55 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [ ]:
cd face_detection

/content/face_detection


In [ ]:
!git lfs pull

In [ ]:
pip install -e .

Obtaining file:///content/face_detection
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
cd ..

/content


In [ ]:
!git clone https://github.com/hhj1897/face_alignment.git

Cloning into 'face_alignment'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 190 (delta 27), reused 27 (delta 26), pack-reused 158
Receiving objects: 100% (190/190), 213.82 MiB | 12.92 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Updating files: 100% (14/14), done.


In [ ]:
cd face_alignment

/content/face_alignment


In [ ]:
pip install -e .

Obtaining file:///content/face_alignment
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ibug-face-alignment


In [ ]:
cd ..

/content


In [ ]:
!git clone --recurse-submodules https://github.com/hhj1897/face_pose_augmentation.git

Cloning into 'face_pose_augmentation'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 479 (delta 8), reused 26 (delta 6), pack-reused 441
Receiving objects: 100% (479/479), 13.27 MiB | 13.68 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Submodule 'ibug/face_pose_augmentation/tddfa' (https://github.com/cleardusk/3DDFA.git) registered for path 'ibug/face_pose_augmentation/tddfa'
Cloning into '/content/face_pose_augmentation/ibug/face_pose_augmentation/tddfa'...
remote: Enumerating objects: 797, done.        
remote: Counting objects: 100% (6/6), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 797 (delta 2), reused 2 (delta 0), pack-reused 791        
Receiving objects: 100% (797/797), 81.43 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (427/427), done.
Submodule path 'ibug/face_pose_augmentation/tddfa': checked out 'd5c1f6a647a89070b1f9ea4e88

In [ ]:
cd face_pose_augmentation

/content/face_pose_augmentation


In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.1 MB/s eta 0:00:00


In [ ]:
pip install -e .

Obtaining file:///content/face_pose_augmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ibug-face-pose-augmentation


In [ ]:
cd ..

/content


#Pose Augmentation

In [ ]:
from face_alignment.ibug.face_alignment import FANPredictor
from face_detection.ibug.face_detection import RetinaFacePredictor
from face_alignment.ibug.face_alignment.utils import plot_landmarks
from face_pose_augmentation.ibug.face_pose_augmentation import TDDFAPredictor, FacePoseAugmentor

def detect_face(
    image: np.ndarray,
    face_detector: RetinaFacePredictor,
    landmark_detector: FANPredictor,
) -> Tuple[Union[np.ndarray, None], Union[np.ndarray, None]]:
    # Face and landmark detection
    faces = face_detector(image, rgb=False)
    landmarks, scores = landmark_detector(image, faces, rgb=False)

    # Try to select a face
    face_sizes = [(bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) if scs.min() >= 0.2 else -1
                    for bbox, scs in zip(faces, scores)]
    selected_face = np.argmin(face_sizes) if len(face_sizes) > 0 and min(face_sizes) > 0 else -1

    return landmarks[selected_face] if selected_face >= 0 else None


def augment_face_pose(
    tddfa: TDDFAPredictor,
    augmentor: FacePoseAugmentor,
    image: np.ndarray,
    landmarks: np.ndarray,
    delta_pose: np.ndarray, # poses are in degrees
) -> int:
    # Apply 3DDFA
    tddfa_result = TDDFAPredictor.decode(tddfa(image, landmarks, rgb=False))[0]
    pitch, yaw, roll = np.array([tddfa_result['face_pose'][k] for k in ('pitch', 'yaw', 'roll')]) / np.pi * 180.0

    # Determine delta poses
    delta_pose = delta_pose / 180.0 * np.pi

    # Pose augmentation
    augmentation_results = augmentor(image, tddfa_result, delta_pose, landmarks)

    return augmentation_results[0]


In [ ]:
face_detector = RetinaFacePredictor(
    threshold=0.99, device='cuda:0',
    model=RetinaFacePredictor.get_model('resnet50'))

# Create a facial landmark detector
landmark_detector = FANPredictor(
    device='cuda:0', model=FANPredictor.get_model('2dfan2_alt'))

tddfa = TDDFAPredictor(device='cuda:0')
print('3DDFA initialised.')
augmentor = FacePoseAugmentor()
print('Face pose augmentor created.')


3DDFA initialised.
Face pose augmentor created.


##감정별 적용

In [ ]:
i=1
image_folder='/content/data/anger_crop'
image_files=sorted(glob.glob(os.path.join(image_folder,'*')))
output_folder='/content/train/anger'
os.makedirs(output_folder, exist_ok=True)

delta_poses = np.array([
    # [[20, 0, 0]],
    [[-20, 0, 0]],
    [[0, 20, 0]],
    [[0, -20, 0]],
    [[0, 0, 20]],
    # [[0, 0, -20]]
])  # pitch, yaw, roll 순서
for delta_pose in delta_poses:
  for image_fi in tqdm(image_files):
      try:
        image = cv2.imread(image_fi)
      except:
        print('이미지 로드 실패', image_fi)
        break
        continue

      landmarks = detect_face(image, face_detector, landmark_detector)
      if landmarks is None:
        print('Failed to detect landmarks from ',image_fi)
        continue

      result = augment_face_pose(tddfa, augmentor, image, landmarks, delta_pose)

      if "correspondence_map" in result:
        del result["correspondence_map"]

      filename=f'anger{i:05}.jpg'
      warped_image = result["warped_image"].copy()
      cv2.imwrite(os.path.join(output_folder, filename), warped_image)
      del result["warped_image"]
      # pickle.dump(result, open(os.path.join(output_folder, Path(filename).stem+'.pkl'),'wb'))
      i+=1

  print(f'{delta_pose} done.')

In [ ]:
i=1
image_folder='/content/drive/MyDrive/train/happy'
image_files=sorted(glob.glob(os.path.join(image_folder,'*')))
output_folder='/content/drive/MyDrive/aug/happy'
os.makedirs(output_folder, exist_ok=True)

delta_poses = np.array([
    [[20, 0, 0]],
    [[-20, 0, 0]],
    # [[0, 20, 0]],
    [[0, -20, 0]],
    [[0, 0, 20]],
    # [[0, 0, -20]]
])  # pitch, yaw, roll 순서
for delta_pose in delta_poses:
  for j,image_fi in enumerate(tqdm(image_files)):
      try:
          image = cv2.imread(image_fi)
      except:
          print('이미지 로드 실패', image_fi)
          continue

      landmarks = detect_face(image, face_detector, landmark_detector)
      if landmarks is None:
          print('Failed to detect landmarks from ',image_fi)
          continue

      result = augment_face_pose(tddfa, augmentor, image, landmarks, delta_pose)

      if "correspondence_map" in result:
          del result["correspondence_map"]

      filename=f'happy{i:05}.jpg'
      warped_image = result["warped_image"].copy()
      cv2.imwrite(os.path.join(output_folder, filename), warped_image)
      del result["warped_image"]
      # pickle.dump(result, open(os.path.join(output_folder, Path(filename).stem+'.pkl'),'wb'))
      i+=1
  print(f'{delta_pose} done')

print('All done.')

In [ ]:
i=1
image_folder='/content/drive/MyDrive/train/panic'
image_files=sorted(glob.glob(os.path.join(image_folder,'*')))
output_folder='/content/drive/MyDrive/aug/panic'
os.makedirs(output_folder, exist_ok=True)

delta_poses = np.array([
    [[20, 0, 0]],
    [[-20, 0, 0]],
    # [[0, 20, 0]],
    [[0, -20, 0]],
    [[0, 0, 20]],
    # [[0, 0, -20]]
])  # pitch, yaw, roll 순서
for delta_pose in delta_poses:
  for image_fi in tqdm(image_files):
      try:
          image = cv2.imread(image_fi)
      except:
          print('이미지 로드 실패', image_fi)
          continue

      landmarks = detect_face(image, face_detector, landmark_detector)
      if landmarks is None:
          print('Failed to detect landmarks from ',image_fi)
          continue

      result = augment_face_pose(tddfa, augmentor, image, landmarks, delta_pose)

      if "correspondence_map" in result:
          del result["correspondence_map"]

      filename=f'panic{i:05}.jpg'
      warped_image = result["warped_image"].copy()
      cv2.imwrite(os.path.join(output_folder, filename), warped_image)
      del result["warped_image"]
      # pickle.dump(result, open(os.path.join(output_folder, Path(filename).stem+'.pkl'),'wb'))
      i+=1
  print(f'{delta_pose} done')

print('All done.')

In [ ]:
i=1
image_folder='/content/drive/MyDrive/train/sadness'
image_files=sorted(glob.glob(os.path.join(image_folder,'*')))
output_folder='/content/drive/MyDrive/aug/sadness'
os.makedirs(output_folder, exist_ok=True)

delta_poses = np.array([
    [[20, 0, 0]],
    [[-20, 0, 0]],
    # [[0, 20, 0]],
    [[0, -20, 0]],
    [[0, 0, 20]],
    # [[0, 0, -20]]
])  # pitch, yaw, roll 순서
for delta_pose in delta_poses:
  for image_fi in tqdm(image_files):
      try:
          image = cv2.imread(image_fi)
      except:
          print('이미지 로드 실패', image_fi)
          continue

      landmarks = detect_face(image, face_detector, landmark_detector)
      if landmarks is None:
          print('Failed to detect landmarks from ',image_fi)
          continue

      result = augment_face_pose(tddfa, augmentor, image, landmarks, delta_pose)

      if "correspondence_map" in result:
          del result["correspondence_map"]

      filename=f'sadness{i:05}.jpg'
      warped_image = result["warped_image"].copy()
      cv2.imwrite(os.path.join(output_folder, filename), warped_image)
      del result["warped_image"]
      # pickle.dump(result, open(os.path.join(output_folder, Path(filename).stem+'.pkl'),'wb'))
      i+=1
  print(f'{delta_pose} done')

print('All done.')